<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/api_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 1. Instalación de dependencias de Python y descarga de ngrok
print("Instalando dependencias de Python y descargando ngrok...")
!pip install fastapi uvicorn pydantic joblib nest_asyncio pyngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/ngrok
print("Dependencias instaladas y ngrok configurado.")

# 2. Subir el modelo y el escalador
from google.colab import files

print("Por favor, suba los archivos 'modelo_riesgo_crediticio.pkl' y 'scaler.pkl'.")
uploaded = files.upload()

# 3. Definir la API con FastAPI
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

# Definir la estructura de los datos de entrada
class CreditRiskInput(BaseModel):
    person_home_ownership: int
    loan_intent: int
    loan_grade: int
    cb_person_default_on_file: int
    # Añade aquí otros campos si los hay

# Inicializar la aplicación FastAPI
app = FastAPI()

# Cargar el modelo y el escalador al inicio
model = joblib.load('modelo_riesgo_crediticio.pkl')
scaler = joblib.load('scaler.pkl')

# Definir el endpoint /predict
@app.post("/predict")
def predict(input_data: CreditRiskInput):
    # Convertir los datos de entrada a un array numpy
    input_dict = input_data.dict()
    input_array = np.array([list(input_dict.values())])

    # Normalizar los datos usando el escalador cargado
    input_scaled = scaler.transform(input_array)

    # Realizar la predicción
    prediction = model.predict(input_scaled)

    # Interpretar la predicción
    result = "Incumplimiento" if prediction[0] == 1 else "Cumplimiento"

    return {"prediction": result}

# 4. Configurar y ejecutar el servidor con Uvicorn y ngrok
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading
import time

# Permitir que múltiples eventos asíncronos se ejecuten en Colab
nest_asyncio.apply()

# Configurar ngrok con tu Authtoken
ngrok_auth_token = "TU_AUTHTOKEN_AQUI"  # Reemplaza con tu Authtoken de ngrok
ngrok.set_auth_token(ngrok_auth_token)

# Abrir un túnel con ngrok en el puerto 8000
public_url = ngrok.connect(8000)
print(f"API está disponible en: {public_url}/predict")

# Función para ejecutar Uvicorn
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Ejecutar Uvicorn en un hilo separado
thread = threading.Thread(target=run)
thread.start()

# Esperar unos segundos para asegurarse de que el servidor esté corriendo
time.sleep(5)

# 5. Probar la API (Opcional)
import requests

# Reemplaza con tu URL pública de ngrok
api_url = f"{public_url}/predict"

# Datos de ejemplo para la predicción
input_data = {
    "person_home_ownership": 2,  # Ejemplo de valor codificado
    "loan_intent": 1,            # Ejemplo de valor codificado
    "loan_grade": 3,             # Ejemplo de valor codificado
    "cb_person_default_on_file": 0  # Ejemplo de valor codificado
    # Añade aquí otros campos si los hay
}

# Realizar la petición POST
response = requests.post(api_url, json=input_data)

# Mostrar la respuesta
print("Respuesta de la API:", response.json())


Instalando dependencias de Python y descargando ngrok...
--2024-10-30 17:35:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  5.92MB/s    in 2.2s    

2024-10-30 17:35:33 (5.92 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Dependencias instaladas y ngrok configurado.
Por favor, suba los archivos 'modelo_riesgo_crediticio.pkl' y 'scaler.pkl'.


Saving modelo_riesgo_crediticio.pkl to modelo_riesgo_crediticio (3).pkl


ERROR:pyngrok.process.ngrok:t=2024-10-30T17:35:45+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: TU_AUTHTOKEN_AQUI\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2024-10-30T17:35:45+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: TU_AUTHTOKEN_AQUI\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: TU_AUTHTOKEN_AQUI\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.